<h1 align="center"> <b>LSTM</b></h1>
<h5 align="center">Les réseaux de neurones - Aurélien Vannieuwenhuyze </h5>

In [ ]:
# Paramètres de prétraitement
SEQ_LEN = 50
SEQ_SKIP = 3

# Paramètres d'apprentissage
NB_ITER = 50
BATCH_SIZE = 64

# Paramètres de la prédiction (longueur du paragraphe à prédire, séquence de départ)
PRED_LEN = 500
PRED_START_INDEX = 1500

# Paramètres visualisation
TAR_LAYER = 6   # 1-indexed
TAR_CELL = 0    # 0-indexed

In [ ]:
import numpy as np


def get_alphabet(text):
    return sorted(list(set(text)))


def create_dicts(alphabet):
    i = 0
    index_char = {}
    char_index = {}
    for c in alphabet:
        char_index[c] = i
        index_char[i] = c
        i+=1
    return char_index, index_char


def encode(text, char_index, nb_chars, data_type):
    dataset = []
    for c in text:
        code = np.zeros(nb_chars, dtype=data_type)
        code[char_index[c]] = 1
        dataset.append(code)
    return np.array(dataset)


def decode(dataset, index_char):
    txt = ""
    for char in dataset:
        i = np.argmax(char)
        c = index_char[i]
        txt = txt + c
    return txt


def create_data(dataset, seq_length, skip):
    inputs, targets = [], []
    for i in range(0,len(dataset)-seq_length,skip):
        x = list(dataset[i:(i+seq_length)])
        y = dataset[i+seq_length]
        inputs.append(x)
        targets.append(y)
    return np.array(inputs), np.array(targets)


def to_array(texte, line_length):
    res = []
    for i in range(0,len(texte),line_length):
        x = list(texte[i:(i+line_length)])
        res.append(x)
    res = np.array(res)
    return res

In [ ]:
# Texte initial
TEXT = open("TEXTES.txt").read().lower()[0:1500000]

In [ ]:
TEXT


'the project gutenberg ebook of la dernière lettre écrite par des soldats\nfrançais tombés au champ d\'honneur 1914-1918, by l\'union des pères et\ndes mères dont les fils sont morts pour la patrie\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.org\n\n\ntitle: la dernière lettre écrite par des soldats français\n       tombés au champ d\'honneur 1914-1918\n\nauthor: l\'union des pères et des mères dont les fils sont morts pour la patrie\n\nrelease date: may 21, 2004 [ebook #12401]\n\nlanguage: french\n\n\n*** start of this project gutenberg ebook la derniere lettre ecrite ***\n\n\n\n\nproduced by tonya allen and pg distributed proofreaders\n\n\n\n\nla dernière lettre\n\nécrite par des soldats français\n\ntombés au champ d\'honneur\n\n1914-1918\n\n  ces lettres ont été choisies\n  par de

In [ ]:
# Ensemble des caractères distincts du texte
ALPHABET = get_alphabet(TEXT)
ALPHABET

['\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '«',
 '°',
 '»',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û']

In [ ]:
# Nombre de caractères distincts
NB_CHARS = len(ALPHABET)

In [ ]:
NB_CHARS

73

In [ ]:
# Dictionnaires utilisés pour l'encodage et le décodage
CHAR_INDEX, INDEX_CHAR = create_dicts(ALPHABET)

In [ ]:
CHAR_INDEX

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '«': 58,
 '°': 59,
 '»': 60,
 'à': 61,
 'â': 62,
 'ç': 63,
 'è': 64,
 'é': 65,
 'ê': 66,
 'ë': 67,
 'î': 68,
 'ï': 69,
 'ô': 70,
 'ù': 71,
 'û': 72}

In [ ]:
# Texte encodé (one-hot encoding de chaque caractère du texte)
ENCODED_TEXT = encode(TEXT, CHAR_INDEX, NB_CHARS, bool)

In [ ]:
ENCODED_TEXT

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [ ]:
# Données d'entraînement
# Un input est une séquences de caractères encodés
# Un target est le caractère suivant cette séquence (la valeur à prédire)
INPUTS, TARGETS = create_data(ENCODED_TEXT, SEQ_LEN, SEQ_SKIP)

In [ ]:
INPUTS

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False]],

       [[False,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
# Phrase de départ de la prédiction
FIRST_SENTENCE = "etait le 23 septembre apres-midi ensoleillee "

# RESEAU

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Activation, Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
# Définition de checkpoints
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1,save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(SEQ_LEN, NB_CHARS), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(NB_CHARS))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(INPUTS, TARGETS, batch_size=BATCH_SIZE, epochs=NB_ITER, callbacks=callbacks_list, verbose=1)

Epoch 1/50
  12/1313 [..............................] - ETA: 13:57 - loss: 3.7400

KeyboardInterrupt: ignored

# PREDICTIONS

In [ ]:
# Sampling dans la distribution de probabilités prédite
def sample(preds, temperature=1.0, do_sample=True):
    # Avec sampling
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    # Sans sampling
    preds = np.reshape(preds, (1, preds.shape[0]))
    if do_sample:
        return np.argmax(probas)
    else:
        return np.argmax(preds)


def predict_single_input(model, sentence):
    x = encode(sentence, CHAR_INDEX, NB_CHARS, float)
    x = np.reshape(x, (1, x.shape[0], x.shape[1]))
    preds = model.predict(x, verbose=0)[0]
    if len(preds.shape) > 1:
        return preds[-1]
    else:
        return preds


# Génération d'un paragraphe commençant par FIRST_SEQUENCE
def predict_paragraph(pred_len, model, do_sample=True, temperature=1.0):
    first_s = FIRST_SENTENCE
    generated = first_s
    for i in range(pred_len):
        preds = predict_single_input(model, first_s)
        next_index = sample(preds, temperature, do_sample)
        next_char = INDEX_CHAR[next_index]
        generated += next_char
        first_s = first_s[1:] + next_char
    return generated


# Récupération des activations (valeurs) d'une couche donnée
def predict_activations(complete_model, predicted_paragraph, tar_layer):
    activations = []
    # Création du modèle tronqué
    partial_model = Sequential()
    for l in range(tar_layer):
        partial_model.add(complete_model.layers[l])
    partial_model.compile(loss='categorical_crossentropy', optimizer='adam')
    # Récupération des outputs de la couche à chaque caractère prédit
    for i in range(len(predicted_paragraph)-SEQ_LEN):
        sentence = predicted_paragraph[i:i+SEQ_LEN]
        act = predict_single_input(partial_model, sentence)
        activations.append(act)
    activations = np.transpose(activations)
    fs_l_acts = [[0] * SEQ_LEN + list(a) for a in activations]
    return np.array(fs_l_acts)


# Récupération de toutes les activations du réseau
def all_activations(complete_model, predicted_paragraph):
    net_acts = {}
    for layer in range(1, 1 + len(MODEL.layers)):
        net_acts[layer] = predict_activations(MODEL, predicted_paragraph, layer)
    return net_acts

In [ ]:
from keras.models import Sequential, load_model

In [ ]:
MODEL = load_model('weights-improvement-50-0.4728.hdf5')

In [ ]:
# Phrase de départ de la prédiction
FIRST_SENTENCE = "le sacrifice de tous les soldats tombés pour la dé"

In [ ]:
paragraph = predict_paragraph(PRED_LEN, MODEL, True, 0.2)
print(paragraph)

le sacrifice de tous les soldats tombés pour la décense, dans la pravaile chasse, le bon dieu de mon coeur de faire un peu de
prouver vous recevez cette lettre, ce n'est pas que se défendre que moi. n'est pas la paille que j'ai un peu toujours bon devoir s'avoir vous la france.

je veux par la plus de mon devoir sur la faire un moten devoir sans récombatten, à l'est ce sera une consoleur. car l'he vous et à partouter de tout mon coeur.

parle, de la jamais de notre chère patrie!

ma chère mère, si je meurs en pleurer pation, les bris de ma mama
